In [2]:
try:
    import evidently
except:
    !pip install git+https://github.com/evidentlyai/evidently.git
import numpy as np

from sklearn import datasets
from sklearn import ensemble

from evidently.options import ColorOptions
from evidently.test_suite import TestSuite
from evidently.tests import *

C:\Users\andre\miniconda3\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<b>Prepare Datasets</b>

In [3]:
#Dataset for Data Quality and Integrity
adult_data = datasets.fetch_openml(name='adult', version=2, as_frame='auto')
adult = adult_data.frame

adult_ref = adult[~adult.education.isin(['Some-college', 'HS-grad', 'Bachelors'])]
adult_cur = adult[adult.education.isin(['Some-college', 'HS-grad', 'Bachelors'])]

adult_cur.iloc[:2000, 3:5] = np.nan

C:\Users\andre\miniconda3\lib\site-packages\sklearn\datasets\_openml.py:932: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [4]:
#Dataset for Regression
housing_data = datasets.fetch_california_housing(as_frame='auto')
housing = housing_data.frame

housing.rename(columns={'MedHouseVal': 'target'}, inplace=True)
housing['prediction'] = housing_data['target'].values + np.random.normal(0, 3, housing.shape[0])

housing_ref = housing.sample(n=5000, replace=False)
housing_cur = housing.sample(n=5000, replace=False)

In [5]:

#Dataset for binary label and probabilistic classifcation
bcancer_data = datasets.load_breast_cancer(as_frame='auto')
bcancer = bcancer_data.frame

bcancer_ref = bcancer.sample(n=300, replace=False)
bcancer_cur = bcancer.sample(n=200, replace=False)

bcancer_label_ref = bcancer_ref.copy(deep=True)
bcancer_label_cur = bcancer_cur.copy(deep=True)

model = ensemble.RandomForestClassifier(random_state=1, n_estimators=10)
model.fit(bcancer_ref[bcancer_data.feature_names.tolist()], bcancer_ref.target)

bcancer_ref['prediction'] = model.predict_proba(bcancer_ref[bcancer_data.feature_names.tolist()])[:, 1]
bcancer_cur['prediction'] = model.predict_proba(bcancer_cur[bcancer_data.feature_names.tolist()])[:, 1]

bcancer_label_ref['prediction'] = model.predict(bcancer_label_ref[bcancer_data.feature_names.tolist()])
bcancer_label_cur['prediction'] = model.predict(bcancer_label_cur[bcancer_data.feature_names.tolist()])

In [6]:
#Dataset for Multiclass Classifcation (labels)
iris_data = datasets.load_iris(as_frame='auto')
iris = iris_data.frame

iris_ref = iris.sample(n=75, replace=False)
iris_cur = iris.sample(n=75, replace=False)

model = ensemble.RandomForestClassifier(random_state=1, n_estimators=3)
model.fit(iris_ref[iris_data.feature_names], iris_ref.target)

iris_ref['prediction'] = model.predict(iris_ref[iris_data.feature_names])
iris_cur['prediction'] = model.predict(iris_cur[iris_data.feature_names])

<b>How to run TestSuites?</b>
<br>
<em>Data Intergrity Tests</em>

In [8]:
#dataset-level tests
data_integrity_dataset_tests = TestSuite(tests=[
    TestNumberOfColumns(),
    TestNumberOfRows(),
    TestNumberOfMissingValues(),
    TestShareOfMissingValues(),
    TestNumberOfColumnsWithMissingValues(),
    TestNumberOfRowsWithMissingValues(),
    TestShareOfColumnsWithMissingValues(),
    TestShareOfRowsWithMissingValues(),
    TestNumberOfDifferentMissingValues(),
    TestNumberOfConstantColumns(),
    TestNumberOfEmptyRows(),
    TestNumberOfEmptyColumns(),
    TestNumberOfDuplicatedRows(),
    TestNumberOfDuplicatedColumns(),
    TestColumnsType(),
    
])

data_integrity_dataset_tests.run(reference_data=adult_ref, current_data=adult_cur)
data_integrity_dataset_tests

In [9]:
#column-level tests
data_integrity_column_tests = TestSuite(tests=[
    TestColumnNumberOfMissingValues(column_name='education'),
    TestColumnShareOfMissingValues(column_name='education'),
    TestColumnNumberOfDifferentMissingValues(column_name='education'),
    TestColumnAllConstantValues(column_name='education'),
    TestColumnAllUniqueValues(column_name='education'),
    TestColumnRegExp(column_name='education',reg_exp='^[0..9]')
])

data_integrity_column_tests.run(reference_data=adult_ref, current_data=adult_cur)
data_integrity_column_tests

In [ ]:
#test in a JSON format
data_integrity_dataset_tests.json()

In [ ]:
#tests as a python object
data_integrity_dataset_tests.as_dict()

<em>Data Quality Tests</em>

In [ ]:
#dataset-level tests
data_quality_dataset_tests = TestSuite(tests=[
    TestConflictTarget(),
    TestConflictPrediction(),
    TestTargetPredictionCorrelation(),
    TestHighlyCorrelatedColumns(),
    TestTargetFeaturesCorrelations(),
    TestPredictionFeaturesCorrelations(),
    TestCorrelationChanges(),
])

data_quality_dataset_tests.run(reference_data=iris_ref, current_data=iris_cur)
data_quality_dataset_tests

In [ ]:
#column-level tests
data_quality_column_tests = TestSuite(tests=[
    TestColumnValueMin(column_name='education-num'),
    TestColumnValueMax(column_name='education-num'),
    TestColumnValueMean(column_name='education-num'),
    TestColumnValueMedian(column_name='education-num'),
    TestColumnValueStd(column_name='education-num'),
    TestNumberOfUniqueValues(column_name='education'),
    TestUniqueValuesShare(column_name='education'),
    TestMostCommonValueShare(column_name='education'),
    TestMeanInNSigmas(column_name='education-num'),
    TestValueRange(column_name='education-num'),
    TestNumberOfOutRangeValues(column_name='education-num'),
    TestShareOfOutRangeValues(column_name='education-num'),
    TestValueList(column_name='education'),
    TestNumberOfOutListValues(column_name='education'),
    TestShareOfOutListValues(column_name='education'),
    TestColumnQuantile(column_name='education-num', quantile=0.25),
    TestShareOfOutListValues(column_name='education-num'),
])

data_quality_column_tests.run(reference_data=adult_ref, current_data=adult_cur)
data_quality_column_tests

<em>Data Drift Tests</em>

In [11]:
#dataset-level tests
data_drift_dataset_tests = TestSuite(tests=[
    TestNumberOfDriftedColumns(),
    TestShareOfDriftedColumns(),
])

data_drift_dataset_tests.run(reference_data=adult_ref, current_data=adult_cur)
data_drift_dataset_tests

In [12]:
#column-level tests
data_drift_column_tests = TestSuite(tests=[
    TestColumnDrift(column_name='education-num', stattest='psi', stattest_threshold=0.3)
])

data_drift_column_tests.run(reference_data=adult_ref, current_data=adult_cur)
data_drift_column_tests

<em>Regression Performance Tests</em>

In [13]:
#dataset-level tests
regression_performance_dataset_tests = TestSuite(tests=[
    TestValueMAE(),
    TestValueRMSE(),
    TestValueMeanError(),
    TestValueMAPE(),
    TestValueAbsMaxError(),
    TestValueR2Score()
])

regression_performance_dataset_tests.run(reference_data=housing_ref, current_data=housing_cur)
regression_performance_dataset_tests

<em>Classification Performance Tests</em>

In [14]:
#dataset-level tests
classification_performance_dataset_tests = TestSuite(tests=[
    TestAccuracyScore(),
    TestPrecisionScore(),
    TestRecallScore(),
    TestF1Score(),
    TestPrecisionByClass(label=0),
    TestPrecisionByClass(label=1),
    TestPrecisionByClass(label=2),
    TestRecallByClass(label=0),
    TestRecallByClass(label=1),
    TestRecallByClass(label=2),
    TestF1ByClass(label=0),
    TestF1ByClass(label=1),
    TestF1ByClass(label=2),
])

classification_performance_dataset_tests.run(reference_data=iris_ref, current_data=iris_cur)
classification_performance_dataset_tests

<em>Probabilistic Classification Performance Tests</em>

In [15]:
#dataset-level tests
prob_classification_performance_dataset_tests = TestSuite(tests=[
    TestAccuracyScore(),
    TestPrecisionScore(),
    TestRecallScore(),
    TestF1Score(),
    TestRocAuc(),
    TestLogLoss(),
    TestPrecisionByClass(label=0),
    TestPrecisionByClass(label=1),
    TestRecallByClass(label=0),
    TestRecallByClass(label=1),
    TestF1ByClass(label=0),
    TestF1ByClass(label=1),

])

prob_classification_performance_dataset_tests.run(reference_data=bcancer_ref, current_data=bcancer_cur)
prob_classification_performance_dataset_tests

<em>How to set test parameters?</em>

In [16]:
#simple test parameters
feature_level_tests = TestSuite(tests=[
    TestMeanInNSigmas(column_name='hours-per-week', n_sigmas=3),
    TestShareOfOutRangeValues(column_name='hours-per-week', lte=0),
    TestColumnShareOfMissingValues(column_name='education', lt=0.2),
])

feature_level_tests.run(reference_data=adult_ref, current_data=adult_cur)
feature_level_tests

In [17]:
#options
color_scheme = ColorOptions(
    primary_color="#5a86ad",
    fill_color="#fff4f2",
    zero_line_color="#016795",
    current_data_color= "#c292a1" ,
    reference_data_color="#017b92",
)

In [18]:
data_drift_column_tests = TestSuite(tests=[
    TestColumnDrift(column_name='education-num', stattest='psi')
], options=[color_scheme])

data_drift_column_tests.run(reference_data=adult_ref, current_data=adult_cur)
data_drift_column_tests